In [49]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

import numpy as np

import warnings
warnings.filterwarnings("ignore")

import math
import Config
import datetime
import time
import copy
import BS
from colorama import Fore, Back, Style
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay, precision_score
from scipy.optimize import bisect
from scipy import interpolate

from pymongo import MongoClient
client=MongoClient(Config.DB_Hostname,Config.DB_Port)

def IVOL(C,S,T,K,opt_type):
    
    try:
        def Calc_Call(sig):
            return BS.bs_call(S,K,T/252,Config.interest_rate,sig)-C

        def Calc_Put(sig):
            return BS.bs_put(S,K,T/252,Config.interest_rate,sig)-C

        sigma = 0.5
        if opt_type=="CE":

            while BS.bs_call(S,K,T/252,Config.interest_rate,sigma)>C:
                sigma /= 2

            while BS.bs_call(S,K,T/252,Config.interest_rate,sigma)<C:
                sigma *= 2

            hi = sigma
            lo = hi/2
            return bisect(Calc_Call,lo,hi)    

        else:

            while BS.bs_put(S,K,T/252,Config.interest_rate,sigma)>C:
                sigma /= 2

            while BS.bs_put(S,K,T/252,Config.interest_rate,sigma)<C:
                sigma *= 2

            hi = sigma
            lo = hi/2
            try:
                return bisect(Calc_Put,lo,hi)
            except:
                return np.nan
    
    except Exception as e:
        return np.nan
    
class Gap_Move_Live_Command:
    
    def __init__(self, params):
        
        #params = {"date_today":, "date_tomorrow":, "underlying":, "holding_period":}
        self.params = copy.deepcopy(params)
        
        self.underlying_dynamics = {} #{"NIFTY" : pd.DataFrame()}
        self.vol_surface = {} #{"NIFTY" : pd.DataFrame()}
        self.days_to_expiry = {} #{"NIFTY" : pd.DataFrame()}
        self.expiry_map = {} #{"2023-12-09" : 8}
        self.instrument_list = None
        self.XGBoost_Parameters = {} #{"NIFTY" : {}}
        
        self.data_matrix = {} #{"NIFTY" : pd.DataFrame()}
        self.vol_instruments = None
        self.price_packet = {}
        self.vol_dict = {}
        self.backup_vol = {"NIFTY":0.13, "BANKNIFTY": 0.25}
        self.underlying_future_map = {} #{"NIFTY" :21000.95}
        self.prediction = {} #{"NIFTY":1, "BANKNIFTY":0}
        
        start = time.time()
        self._get_underlying_dynamics()
        print(f"Prices Downloaded in {time.time() - start} seconds")

        start=time.time()
        self._get_vol_surface()
        print(f"Vol Surface Downloaded in {time.time() - start} seconds")

        start=time.time()
        self._get_instrument_list()
        print(f"XGB Params Downloaded in {time.time()-start} seconds")

        start=time.time()
        self._get_days_to_expiry()
        self._get_expiry_map()
        print(f"Expiry Dates Downloaded in {time.time()-start} seconds")
        
        start=time.time()
        self._get_XGB_params()
        print(f"XGB Params Downloaded in {time.time()-start} seconds")
        
    
    def _get_underlying_dynamics(self):

        for underlying in self.params["underlying"]:
            
            temp = pd.DataFrame(client[f'{Config.Data_DB}'][f'{underlying}OHLC'].find())
            temp.drop(columns = ['_id'], inplace = True)
            temp.sort_values(['date', 'batch_id'], inplace = True)
            temp.batch_id = temp.batch_id.astype(int)
            self.underlying_dynamics[underlying] = temp.set_index('date')
    
    def _get_vol_surface(self):
        
        for underlying in self.params["underlying"]:
            
            temp = pd.DataFrame(client[f'{Config.Data_DB}']['Vol_Surface'].find({"SYMBOL" : underlying}))
            temp.drop(columns=['_id', 'CONTRACTS', 'VAL_INLAKH','OPEN_INT', 'CHG_IN_OI', 'SYMBOL'], inplace=True)
            temp.sort_values('date',inplace=True)
            self.vol_surface[underlying] = temp.set_index('date')

    def _get_instrument_list(self):
        
        self.instrument_list = pd.DataFrame(client.Live_Trading.Instrument_Universe.find())
        if len(self.instrument_list)==0 or len(set(self.params["underlying"] + self.instrument_list.underlying.tolist())) != len(self.params["underlying"]):
            raise RuntimeError(Fore.RED + "Empty / Improper Instrument List")
        
        self.instrument_list.drop(columns=['_id'], inplace = True)
        self.instrument_list.instrument_token = self.instrument_list.instrument_token.astype(int)
        self.instrument_list.exchange_token = self.instrument_list.exchange_token.astype(int)
        self.instrument_list.tradingsymbol = self.instrument_list.tradingsymbol.astype(str)
        self.instrument_list.underlying = self.instrument_list.underlying.astype(str)
        self.instrument_list.expiry = self.instrument_list.expiry.astype(str)
        self.instrument_list.strike = self.instrument_list.strike.astype(float)
        self.instrument_list.tick_size = self.instrument_list.tick_size.astype(float)
        self.instrument_list.lot_size = self.instrument_list.lot_size.astype(int)
        self.instrument_list.instrument_type = self.instrument_list.instrument_type.astype(str)
        self.instrument_list.exchange = self.instrument_list.exchange.astype(str)

    def _get_days_to_expiry(self):
        
        for underlying in self.params["underlying"]:
            
            temp = pd.DataFrame(client[f'{Config.Data_DB}']['Days_To_Expiry'].find({"underlying":underlying}))
            temp.drop(columns = ['_id','underlying'],inplace=True)
            temp.sort_values('date',inplace=True)
            self.days_to_expiry[underlying] = temp.set_index('date')

    def _get_expiry_map(self):
        
        days_to_expiry = pd.DataFrame(client.Strategy.Days_To_Expiry.find({"date" : self.params['date_today']})).drop(columns=['_id','date']).to_dict('records')
        days_to_expiry = {temp['underlying']: {'current_week': temp['current_week'], 'next_week': temp['next_week']} for temp in days_to_expiry}

        for underlying in self.params["underlying"]:

            opt_expiry = sorted(self.instrument_list[(self.instrument_list.underlying == underlying)&(self.instrument_list.instrument_type == "CE")].expiry.unique())
            
            if len(opt_expiry) < 2:
                print(Fore.RED, f"Couldnt Find Two Consequtive Expiries For {underlying}")
                print(Style.RESET_ALL)
                continue
                
            self.expiry_map[opt_expiry[0]] = days_to_expiry[underlying]["current_week"]
            self.expiry_map[opt_expiry[1]] = days_to_expiry[underlying]["next_week"]
    
    def _get_vol_instruments(self):
        
        try:
            
            if self.underlying_future_map is not None or len(self.underlying_future_map) != 0:

                vol_instruments = copy.deepcopy(self.instrument_list)
                vol_instruments["days_to_expiry"] = vol_instruments.expiry.map(self.expiry_map)
                vol_instruments["strike_diff"] = abs(vol_instruments.underlying.map(self.underlying_future_map) - vol_instruments.strike)

                temp_list =[]

                for underlying in self.params["underlying"]:

                    temp = vol_instruments[vol_instruments.underlying == underlying]

                    temp_futures = temp[temp.instrument_type == "FUT"].sort_values('expiry').head(1)

                    temp_options = temp[(temp.instrument_type != "FUT")&(temp.days_to_expiry != 1)]
                    temp_options = temp_options[temp_options.expiry == sorted(temp_options.expiry.unique())[0]]
                    temp_options = temp_options.sort_values('strike_diff').head(70)
                    temp_list.append(pd.concat([temp_futures, temp_options]))

                vol_instruments = pd.concat(temp_list).drop(columns = ['days_to_expiry'])

            else:
                vol_instruments = None
                
        except Exception as e:
            print(Fore.RED, f"{e}, Could Not Filter Vol Instruments")
            print(Style.RESET_ALL)
            vol_instruments = None

        self.vol_instruments = copy.deepcopy(vol_instruments)
        return vol_instruments

    def _get_scaled_parameters(self):
        
        
        scaled_parameters = {}
        for underlying in self.params["underlying"]:
            
            sc = StandardScaler()
            y = self.data_matrix[underlying][['gap_move_flag']].values
            X = self.data_matrix[underlying].drop(columns = ['gap_move', 'gap_move_flag']).values

            X_train = sc.fit_transform(X[:-1])
            X_test = sc.transform(X[-1:])
            y_train = y[:-1]
            y_test = y[-1:]
            
            scaled_parameters[underlying] = (X_train, X_test, y_train, y_test)
            
        return scaled_parameters
  
    def _get_XGB_params(self):
        
        for underlying in self.params["underlying"]:
            
            params = pd.DataFrame(client['Gap_Move_Strategy']['XGBoost_Filter_Params'].find({"underlying" : underlying}))
            params = params[params.date == sorted(params.date.unique())[-1]]
            params.drop(columns = ['_id', 'date', 'underlying', 'accuracy', 'max_accuracy', 'min_accuracy', 'sharpe'], inplace = True)

            params.max_depth = params.max_depth.astype(int)
            params.n_estimators = params.n_estimators.astype(int)
            params.learning_rate = params.learning_rate.astype(float)
            params.reg_lambda = params.reg_lambda.astype(float)
            params.subsample = params.subsample.astype(float)
            params.sample_weight = params.sample_weight.astype(float)
        
            self.XGBoost_Parameters[underlying] = params.to_dict('records')[0]

    def _XGB(self, params):

        try:

            model = XGBClassifier(nthread = -1, 
                                  n_estimators = params['n_estimators'], 
                                  learning_rate = params['learning_rate'], 
                                  max_depth = params['max_depth'], 
                                  reg_lambda = params['reg_lambda'], 
                                  subsample = params['subsample'])

            model.fit(params['X_train'], params['y_train'], 
                      sample_weight = [params['sample_weight'] if y==0 else 1 for y in params['y_train'].ravel()])

            return model.predict(params['X_test'])

        except Exception as e:
            raise RuntimeError(f"{e}, Error in running XGBoost")
    
    def _fill_index(self):
        
        self.underlying_dynamics = {underlying : dynamics[dynamics.index!=self.params["date_today"]] for underlying, dynamics in self.underlying_dynamics.items()}
        index = {}
        tries_left = 5
        while tries_left != 0:

            tries_left -= 1
            try:
                timestamp = datetime.datetime.strptime(self.params["date_today"] + " 09:15:00", "%Y-%m-%d %H:%M:%S").timestamp() + 0.5 * 60 * self.params["holding_period"]
                index_tracker = pd.DataFrame(client.Live_Trading.Index_Tracker.find())
                for underlying in self.params["underlying"]:
                    ltp = index_tracker[index_tracker.underlying == underlying]
                    ltp = ltp[ltp.timestamp <= timestamp ].sort_values('timestamp').tail(1).price.tolist()[0]
                    index[underlying] = ltp
                break

            except Exception as e:

                time.sleep(0.1)
                if tries_left != 0:
                    print(Fore.YELLOW, f"{e}, Could Not Get Index Values, Trying Again (tries_left: {tries_left})")
                    print(Style.RESET_ALL)

                else:                    
                    print(Fore.RED, f"{e}, Could Not Get Index Values, Failed")
                    print(Style.RESET_ALL)
                    index = None
        
        if index is not None:
            for underlying in self.params["underlying"]:
                temp = self.underlying_dynamics[underlying].reset_index()
                temp.loc[-1] = [self.params["date_today"], max(int(self.params["holding_period"] / 2), 1), index[underlying], index[underlying], index[underlying], index[underlying]]
                temp.loc[-2] = [self.params["date_today"], min(376 - int(self.params["holding_period"] / 2), 375), index[underlying], index[underlying], index[underlying], index[underlying]]
                self.underlying_dynamics[underlying] = copy.deepcopy(temp.sort_values(['date', 'batch_id']).set_index('date'))
        
        return index

    def _get_price_packet(self):
        
        price_packet = {}
        tries = 5

        while tries > 0:

            try:
                price_packet_dataframe = pd.DataFrame(client.Live_Trading.Price_Packet.find())
                price_packet_dataframe.to_csv('price_packet_dataframe.csv')
                if len(price_packet_dataframe) == 0:
                    raise RuntimeError("Empty Price Packet")
                else:
                    break

            except Exception as e:
                tries -= 1
                if tries == 0:
                    tries -= 1
                    print(Fore.RED, f"{e} Could Not Get Price Packet From DB")
                    print(Style.RESET_ALL)
                    break

                else:
                    if tries > 2 and tries <= 3:
                        print(Fore.GREEN, f"{e} Problem In Retrieving Price Pakcet From DB, Trying Again (tries_left: {tries})")
                        print(Style.RESET_ALL)

                    elif tries <= 2:
                        print(Fore.YELLOW, f"{e} Problem In Retrieving Price Pakcet From DB, Trying Again (tries_left: {tries})")
                        print(Style.RESET_ALL)

                    time.sleep(1)

        if tries <= 0:
            price_packet = None

        else:

            try:
                price_packet_dataframe = price_packet_dataframe.drop(columns = ['_id']).set_index('instrument').astype(float).fillna(0).reset_index().to_dict('records')
                for packet in price_packet_dataframe:

                    if packet["instrument"] in self.params['underlying']:
                        price_packet[packet["instrument"]] = packet["last_price"]

                    else:
                        price_packet[packet["instrument"]] = {'last_price': packet["last_price"],
                                                              'depth' : {
                                                                  'buy' : [{'price' : packet["buy_price_1"], 'quantity': packet["buy_quantity_1"]}, 
                                                                           {'price' : packet["buy_price_2"], 'quantity': packet["buy_quantity_2"]},
                                                                           {'price' : packet["buy_price_3"], 'quantity': packet["buy_quantity_3"]},
                                                                           {'price' : packet["buy_price_4"], 'quantity': packet["buy_quantity_4"]},
                                                                           {'price' : packet["buy_price_5"], 'quantity': packet["buy_quantity_5"]}],
                                                                  'sell' : [{'price' : packet["sell_price_1"], 'quantity': packet["sell_quantity_1"]}, 
                                                                           {'price' : packet["sell_price_2"], 'quantity': packet["sell_quantity_2"]},
                                                                           {'price' : packet["sell_price_3"], 'quantity': packet["sell_quantity_3"]},
                                                                           {'price' : packet["sell_price_4"], 'quantity': packet["sell_quantity_4"]},
                                                                           {'price' : packet["sell_price_5"], 'quantity': packet["sell_quantity_5"]}]}
                                                             }

            except Exception as e:
                print(Fore.RED, f"{e}, Error In Processing Downloaded Price Packet")
                print(Style.RESET_ALL)
                price_packet = None

        #checking if there is incomplete price packet
        if price_packet is not None:
            for instrument in self.params["underlying"] + self.instrument_list.tradingsymbol.tolist():
                if instrument not in price_packet:
                    price_packet = None
                    break

        self.price_packet = copy.deepcopy(price_packet)
        return price_packet
    
    def _get_futures_price_map(self):
        
        try:
            #future price mapping
            futures_map = {}
            for underlying in self.params["underlying"]:
                futures_map[underlying] = self.instrument_list[(self.instrument_list.instrument_type == "FUT") & (self.instrument_list.underlying == underlying)].sort_values('expiry').tradingsymbol.tolist()[0]
            underlying_future_map = {underlying : self.price_packet[future]["last_price"] for underlying, future in futures_map.items()}
        except Exception as e:
            print(Fore.RED, f"{e} Could Not Get Futures_Price_Map")
            print(Style.RESET_ALL)
            underlying_future_map = None
        
        self.underlying_future_map = copy.deepcopy(underlying_future_map)
        return underlying_future_map
        
    def _get_vol_dict(self, time_to_expiry):
        
        try:
            
            current_instrument_price_map = {}
            #calculating ltp for each option instrument
            for tradingsymbol in self.vol_instruments.tradingsymbol:
                
                try:
                    last_price = self.price_packet[tradingsymbol]['last_price']
                    last_price /= 1
                    if last_price == 0:
                        raise RuntimeError()
                except Exception as e:
                    last_price = None

                try:
                    buy = self.price_packet[tradingsymbol]['depth']['buy'][0]['price']
                    buy /= 1
                    if buy == 0:
                        raise RuntimeError()
                except Exception as e:
                    buy = None

                try:
                    sell = self.price_packet[tradingsymbol]['depth']['sell'][0]['price']
                    sell /= 1
                    if sell == 0:
                        raise RuntimeError()
                except Exception as e:
                    sell = None

                mid = np.nan
                if buy is None or sell is None:
                    mid = last_price
                else:
                    mid = 0.5 * (buy + sell)

                current_instrument_price_map[tradingsymbol] = mid

            #ivol calculation
            options = self.vol_instruments[self.vol_instruments.instrument_type != "FUT"]            
            options["future_price"] = options.underlying.map(self.underlying_future_map)
            options["time_to_expiry"] = options.expiry.map(time_to_expiry)
            options["option_price"] = options.tradingsymbol.map(current_instrument_price_map)
            options['vol'] = options[['option_price', 'future_price', 'time_to_expiry', 'strike', 'instrument_type']].dropna().apply(
                    lambda x : IVOL(x.option_price, x.future_price, x.time_to_expiry, x.strike, x.instrument_type),axis=1)

            # filling missing vols with neighbouring strikes
            tradingsymbol_underlying_map = options.set_index('tradingsymbol').underlying.to_dict()
            tradingsymbol_expiry_map = options.set_index('tradingsymbol').underlying.to_dict()
            options = options.sort_values(['underlying', 'expiry', 'strike', 'instrument_type']).groupby(['underlying', 'expiry']).ffill().bfill()
            options['underlying'] = options.tradingsymbol.map(tradingsymbol_underlying_map)
            options['expiry'] = options.tradingsymbol.map(tradingsymbol_expiry_map)
            options.vol = np.where(options.vol.isna(), options.underlying.map(self.backup_vol), options.vol)

            #saving work
            vol_dict = options[['tradingsymbol', 'vol']].set_index('tradingsymbol').vol.to_dict()
            
        except Exception as e:
            print(Fore.RED, f"{e}, Could Not Calculate Current Vols")
            print(Style.RESET_ALL)
            vol_dict = None
        
        self.vol_dict = copy.deepcopy(vol_dict)
        return vol_dict

    def _fill_vol_surface(self, time_to_expiry):

        try:
            for underlying in self.params["underlying"]:
                temp = self.vol_surface[underlying]
                self.vol_surface[underlying] = temp[temp.index != self.params["date_today"]]

            summary=pd.DataFrame(index=self.params["underlying"],
                                 columns=["curv_tan","expected_return","expected_variance","expected_skew",
                                          "expected_kurt","strike_conc_1st_moment","strike_conc_2nd_moment",
                                          "strike_conc_3rd_moment","strike_conc_4th_moment","vol_neg5","vol_neg4",
                                          "vol_neg3","vol_neg2","vol_neg1","vol_0","vol_pos1","vol_pos2","vol_pos3",
                                          "vol_pos4","vol_pos5"])

            curv_tan=[]
            expected_return=[]
            expected_variance=[]
            expected_skew=[]
            expected_kurt=[]
            vol_surf=[]
            strike_conc_1st_moment=[]
            strike_conc_2nd_moment=[]
            strike_conc_3rd_moment=[]
            strike_conc_4th_moment=[]

            for i in range(11):
                vol_surf.append([])

            for underlying in self.params["underlying"]:

                fut_ref = self.underlying_future_map[underlying]
                temp = self.vol_instruments[(self.vol_instruments.underlying == underlying) & (self.vol_instruments.instrument_type != "FUT")]    
                temp["strike_diff"] = fut_ref - temp.strike    
                temp = pd.concat([temp[(temp.strike_diff >= 0) & (temp.instrument_type == "CE")].sort_values('strike_diff').head(10), temp[(temp.strike_diff <= 0) & (temp.instrument_type == "PE")].sort_values('strike_diff', ascending=False).head(10)])
                temp["IVOL"] = temp.tradingsymbol.map(self.vol_dict)
                T=time_to_expiry[temp.expiry.unique()[0]]
                temp.to_csv('vol_instruments.csv')
                #initialization
                strike_points=[]
                ivols=[]
                fitted_ivols=np.array([])
                fitted_strike=np.array([])
                x_points=[]
                y_points=[]
                pdf=[]
                temp_s=False
                strike_increment=0.1
                strike_lower_bound=-5.1
                strike_upper_bound=5.2

                for s,iv in sorted(zip(temp.strike_diff,temp.IVOL)):

                    if temp_s==True and abs(previous_s-s)<1e-5:
                        ivols[-1]=(ivols[-1]+iv)/2
                        continue
                    if temp_s==False:
                        temp_s=True
                    previous_s=s
                    strike_points.append(-s)
                    ivols.append(iv)

                for s, iv in sorted(zip(strike_points,ivols)):
                    x_points.append(s)
                    y_points.append(iv)

                tck = interpolate.splrep(x_points, y_points)
                fitted_strike = np.arange(strike_lower_bound, strike_upper_bound, strike_increment)
                fitted_ivols = [max(interpolate.splev(s * fut_ref/100, tck), 0.01) for s in fitted_strike]
                curv_tan.append((fitted_ivols[51] - fitted_ivols[49])/((fitted_strike[51] - fitted_strike[49]) / 100))                
                pdf = [BS.bs_call(fut_ref, (1 + (k / 100)) * fut_ref, T / 252, 0.07, iv) for k, iv in zip(fitted_strike, fitted_ivols)]
                pdf = [max((pdf[i + 1] - 2 * pdf[i] + pdf[i - 1]) / ((strike_increment * fut_ref / 100)**2), 0) for i in range(1, len(pdf) - 1)]

                try:
                    pdf=[p/sum(pdf) for p in pdf]
                except Exception as e:
                    try:
                        print(e,"using default pdf")
                        pdf=[2/(len(pdf)-1) if i > (len(pdf)-1)/2 else 0 for i,p in enumerate(pdf)]
                    except Exception as e:
                        print(e,"Check PDF")
                        pdf=[1/len(pdf) for p in pdf]

                fitted_ivols=[float(fitted_ivols[i]) for i in range(1,len(fitted_ivols)-1)]
                fitted_strike=[round(float(fitted_strike[i]),1) for i in range(1,len(fitted_strike)-1)]

                ret=sum([pdf[i]*fitted_strike[i] for i in range(len(fitted_strike))])
                vol=((sum([pdf[i]*((ret-fitted_strike[i])**2)for i in range(len(fitted_strike))]))**0.5)/100
                skew=(sum([pdf[i]*((ret-fitted_strike[i])**3)for i in range(len(fitted_strike))]))

                if skew > 0:
                    skew = math.pow(skew, float(1)/3)/100
                elif skew < 0:
                    skew = -math.pow(abs(skew), float(1)/3)/100
                else:
                    skew = 0

                kurt=((sum([pdf[i]*((ret-fitted_strike[i])**4)for i in range(len(fitted_strike))]))**0.25)/100
                expected_return.append(ret*2.52/T)
                expected_variance.append(vol*((252/T)**0.5))
                expected_skew.append(skew*((252/T)**(1/3)))
                expected_kurt.append(kurt*((252/T)**0.25))

                for i,n in enumerate(np.arange(-5,5.1,1)):
                    for j,k in enumerate(fitted_strike):
                        if abs(k-n)<1e-8:
                            vol_surf[i].append(fitted_ivols[j])

                strike_conc_1st_moment.append(sum([s*iv/100 for s,iv in zip(fitted_strike,fitted_ivols)])/sum(fitted_ivols))
                strike_conc_2nd_moment.append((sum([((s/100)**2)*iv for s,iv in zip(fitted_strike,fitted_ivols)])/sum(fitted_ivols))**0.5)
                skew=sum([((s/100)**3)*iv for s,iv in zip(fitted_strike,fitted_ivols)])/sum(fitted_ivols)
                if skew > 0:
                    skew = math.pow(skew, float(1)/3)
                elif skew < 0:
                    skew = -math.pow(abs(skew), float(1)/3)
                else:
                    skew = 0
                strike_conc_3rd_moment.append(skew)
                strike_conc_4th_moment.append(np.power(sum([((s/100)**4)*iv for s,iv in zip(fitted_strike,fitted_ivols)])/sum(fitted_ivols),0.25))

            summary.curv_tan=curv_tan
            summary.expected_return=expected_return
            summary.expected_variance=expected_variance
            summary.expected_skew=expected_skew
            summary.expected_kurt=expected_kurt
            summary.strike_conc_1st_moment=strike_conc_1st_moment
            summary.strike_conc_2nd_moment=strike_conc_2nd_moment
            summary.strike_conc_3rd_moment=strike_conc_3rd_moment
            summary.strike_conc_4th_moment=strike_conc_4th_moment

            summary["vol_neg5"]=vol_surf[0]
            summary["vol_neg4"]=vol_surf[1]
            summary["vol_neg3"]=vol_surf[2]
            summary["vol_neg2"]=vol_surf[3]
            summary["vol_neg1"]=vol_surf[4]
            summary["vol_0"]=vol_surf[5]
            summary["vol_pos1"]=vol_surf[6]
            summary["vol_pos2"]=vol_surf[7]
            summary["vol_pos3"]=vol_surf[8]
            summary["vol_pos4"]=vol_surf[9]
            summary["vol_pos5"]=vol_surf[10]
            summary.fillna(0.1, inplace=True)

        except Exception as e:
            print(Fore.RED, f"{e}, Could Not Calculate Current Vols")
            print(Style.RESET_ALL)
            summary = None

        if summary is not None:
            for underlying in self.params["underlying"]:
                self.vol_surface[underlying].loc[self.params["date_today"]] = summary.loc[underlying]

        return summary

    def _make_features(self):

        try:

            data_matrix_dict = {}
            for underlying in self.params["underlying"]:

                underlying_dynamics = self.underlying_dynamics[underlying]
                vol_surface = self.vol_surface[underlying]
                vol_surface.loc[self.params["date_tomorrow"]] = [np.nan]*len(vol_surface.columns)

                closing_candle = underlying_dynamics[underlying_dynamics.batch_id==375 - (self.params["holding_period"]*0.5 - 1)][['open']]
                closing_candle.loc[self.params["date_tomorrow"]] = np.nan
                closing_candle=closing_candle.shift()
                opening_candle = underlying_dynamics[underlying_dynamics.batch_id==1 + (self.params["holding_period"]*0.5 - 1)][['close']]
                opening_candle.loc[self.params["date_tomorrow"]] = np.nan
                candle = pd.merge(closing_candle, opening_candle, left_index=True, right_index=True).ffill()

                data_matrix = pd.DataFrame()
                data_matrix["gap_move"] = (candle.close - candle.open) / candle.open
                data_matrix["gap_move_flag"] = np.where(data_matrix.gap_move>=0, 1, 0)
                data_matrix["gap_move_lagged_1"] = data_matrix.gap_move.shift()
                data_matrix["gap_move_lagged_2"] = data_matrix.gap_move.shift(2)
                data_matrix["gap_move_lagged_5"] = data_matrix.gap_move.shift(5)
                data_matrix = pd.merge(data_matrix, vol_surface.shift(), left_index = True, right_index = True).dropna()
                data_matrix_dict[underlying] = data_matrix

        except Exception as e:
            print(Fore.RED, f"{e}, Could Not Make Features")
            print(Style.RESET_ALL)
            data_matrix_dict = None

        self.data_matrix = copy.deepcopy(data_matrix_dict)
        return data_matrix_dict

    def _update_features(self):
        
        success_flag = False
        index_flag = False
        packet_flag = False
        futures_map_flag = False
        
        vol_instruments_flag = False
        vol_flag = False
        vol_surface_flag = False
        feature_flag = False
        
        days_since_start = (datetime.datetime.now().timestamp() - datetime.datetime.strptime(f"{self.params['date_today']}" + " 09:15:00", "%Y-%m-%d %H:%M:%S").timestamp()) / 22500
        time_to_expiry = {expiry_date: days_to_expiry - days_since_start for expiry_date, days_to_expiry in self.expiry_map.items()}
        tries = 10
        
        while tries !=0:
            
            tries-=1
            
            #update index
            if not index_flag:
                index = self._fill_index()
                if index is not None:
                    index_flag = True
                continue
            
            #update price_packet
            if not packet_flag:
                packet = self._get_price_packet()
                if packet is not None:
                    packet_flag = True
                continue

            #update future price map
            if not futures_map_flag:
                futures_map = self._get_futures_price_map()
                if futures_map is not None:
                    futures_map_flag = True
                continue
            
            #update vol_dict
            if not vol_instruments_flag:
                vol_instruments = self._get_vol_instruments()
                if vol_instruments is not None:
                    vol_instruments_flag = True
                continue

            #update vol_dict
            if not vol_flag:
                vol_dict = self._get_vol_dict(time_to_expiry)
                if vol_dict is not None:
                    vol_flag = True
                continue

            #update vol_surface
            if not vol_surface_flag:
                vol_surface = self._fill_vol_surface(time_to_expiry)
                if vol_surface is not None:
                    vol_surface_flag = True
                continue

            #make features
            if not feature_flag:
                data_matrix = self._make_features()
                if data_matrix is not None:
                    feature_flag = True
                continue

            success_flag = True
            break
        
        return success_flag
      
    def _make_prediction(self):
        
        try:
            
            scaled_parameters = self._get_scaled_parameters()
            prediction_dict = {}
            
            for underlying in self.params["underlying"]:
                
                X_train, X_test, y_train, y_test = scaled_parameters[underlying]
                params_XGB = copy.deepcopy(self.XGBoost_Parameters[underlying])
                params_XGB['X_train'] = X_train
                params_XGB['X_test'] = X_test
                params_XGB['y_train'] = y_train
                
                print(self._XGB(params_XGB).ravel())
                prediction_dict[underlying] = self._XGB(params_XGB).ravel()[0]
            
        except Exception as e:
            print(Fore.RED, f"{e}, Could Not Predict, Buying Straddles Instead")
            print(Style.RESET_ALL)
            prediction_dict = {underlying : 0 for underlying in self.params["underlying"]}
        
        self.prediction = copy.deepcopy(prediction_dict)

In [50]:
params = {"date_today": "2024-01-18", 
          "date_tomorrow": "2024-01-19", 
          "underlying": ["NIFTY"], 
          "holding_period":2}

self = Gap_Move_Live_Command(params)

Prices Downloaded in 3.576329231262207 seconds
Vol Surface Downloaded in 0.026561975479125977 seconds
XGB Params Downloaded in 0.0031697750091552734 seconds
Expiry Dates Downloaded in 0.01890110969543457 seconds
XGB Params Downloaded in 0.0016052722930908203 seconds


In [51]:
self._update_features()

True

In [52]:
self._make_prediction()

[1]


In [53]:
self.prediction

{'NIFTY': 1}